##### Copyright 2021 The TensorFlow Federated Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ajustes de agregações recomendadas para aprendizado

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tuning_recommended_aggregators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

O módulo `tff.learning` contém diversas maneiras de agregar atualizações do modelo com a configuração padrão recomendada:

- `tff.learning.robust_aggregator`
- `tff.learning.dp_aggregator`
- `tff.learning.compression_aggregator`
- `tff.learning.secure_aggregator`

Neste tutorial, explicamos a motivação subjacente, como eles são implementados e fornecemos sugestões de como personalizar a configuração deles. 

---

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [1]:
import math
import tensorflow_federated as tff
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

Os métodos de agregação são representados por objetos que podem ser passados para `tff.learning.algorithms.build_weighted_fed_avg` (bem como `build_unweighted_fed_avg`) como seu argumento palavra-chave `model_aggregator`. Dessa forma, os agregadores discutidos aqui podem ser usados diretamente para modificar um [tutorial](federated_learning_for_text_generation.ipynb) [anterior](federated_learning_for_image_classification.ipynb) sobre aprendizado federado.

A média ponderada de referência do algoritmo [FedAvg](http://proceedings.mlr.press/v54/mcmahan17a/mcmahan17a.pdf) pode ser expressada usando `tff.aggregators.MeanFactory` da seguinte forma:

```
mean = tff.aggregators.MeanFactory()
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    ...,
    model_aggregator=mean)
```

As técnicas que podem ser usadas para estender a média ponderada abrangidas por este tutorial são:

- Zeragem
- Corte
- Privacidade diferencial
- Compressão
- Agregação segura

A extensão é feita usando-se composição, em que `MeanFactory` encapsula a fábrica interna, à qual delega parte da agregação, ou ela mesma é encapsulada por outra fábrica de agregação. Confira mais detalhes da concepção no tutorial [Implementando agregadores personalizados](custom_aggregators.ipynb).

Primeiro, vamos explicar como permitir e configurar essas técnicas individualmente e depois vamos mostrar como elas podem ser combinadas.

## Técnicas

Antes de analisarmos as técnicas individuais, vamos primeiro apresentar o algoritmo de correspondência de quantis, que será útil para configurar as técnicas abaixo.

### Correspondência de quantis

Algumas das técnicas de agregação abaixo precisam usar um limite de norma que controle parte dos aspectos de agregação. Esses limites podem ser fornecidos como uma constante, mas geralmente é melhor adaptar o limite durante o treinamento. A forma recomendada é usar o algoritmo de correspondência de quantis de [Andrew et al. (2019)](https://arxiv.org/abs/1905.03871), proposto inicialmente devido à compatibilidade com privacidade diferencial, mas que é útil de forma mais ampla. Para estimar o valor de um determinado quantil, você pode usar `tff.aggregators.PrivateQuantileEstimationProcess`. Por exemplo: para adaptar para a média de uma distribuição, você pode usar:

In [ ]:
median_estimate = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0, target_quantile=0.5, learning_rate=0.2)

Diferentes técnicas que usam o algoritmo de estimativa de quantis exigirão diferentes valores dos parâmetros do algoritmo, como veremos. De forma geral, aumentar o parâmetro `learning_rate` proporciona adaptação mais rápida para o quantil correto, mas com uma variância maior. O método de classe `no_noise` constrói um processo de correspondência de quantis que não adiciona ruído para privacidade diferencial.

### Zeragem

Zeragem refere-se a substituir valores anormalmente grandes por zero. Aqui, "anormalmente grandes" pode significar maiores do que um limiar predefinido ou grandes em relação a valores de rodadas anteriores da computação. A zeragem pode aumentar a robustez do sistema ao corrompimento de dados em clientes com falhas.

Para computar uma média de valores com normas L-infinitas maiores do que `ZEROING_CONSTANT` zerado, encapsulamos uma `tff.aggregators.MeanFactory` em uma `tff.aggregators.zeroing_factory` que faz a zeragem:

```
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=tff.aggregators.MeanFactory())
```

Aqui, encapsulamos uma `MeanFactory` em uma `zeroing_factory` porque queremos que os efeitos (pré-agregação) da `zeroing_factory` se apliquem aos valores nos clientes antes de serem passados para a `MeanFactory` interna para agregação via cálculo de médias.

Entretanto, para a maioria das aplicações, recomendamos a zeragem adaptativa com o estimador de quantil. Para isso, usamos o algoritmo de correspondência de quantis da seguinte forma:

In [ ]:
zeroing_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=10.0,
    target_quantile=0.98,
    learning_rate=math.log(10),
    multiplier=2.0,
    increment=1.0)
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=zeroing_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# zeroing_mean = tff.learning.robust_aggregator(clipping=False)

Os parâmetros foram escolhidos de forma que o processo se adapte muito rapidamente (`learning_rate` relativamente grande) para um valor um pouco maior do que os maiores valores observados até agora. Para uma estimativa de quantil `Q`, o limiar usado para zeragem será `Q * multiplier + increment` (Q * multiplicador + incremento).

### Corte para a norma L2 superior

O corte de atualizações de clientes (projetando em uma bola L2) pode melhorar a robustez a pontos fora da curva. Uma `tff.aggregators.clipping_factory` é estruturada exatamente como a `tff.aggregators.zeroing_factory` discutida acima e pode receber uma constante ou um `tff.templates.EstimationProcess` como argumento `clipping_norm`. A prática recomendada é usar o corte que se adapte de forma moderadamente rápida a uma norma moderadamente alta da seguinte forma:

In [ ]:
clipping_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0,
    target_quantile=0.8,
    learning_rate=0.2)
clipping_mean = tff.aggregators.clipping_factory(
    clipping_norm=clipping_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# clipping_mean = tff.learning.robust_aggregator(zeroing=False)

Em nossa experiência para diversos problemas, o valor preciso de `target_quantile` não parece importar muito, desde que as taxas de aprendizado sejam ajustadas de forma adequada. Entretanto, defini-lo como um valor muito baixo pode exigir o aumento da taxa de aprendizado do servidor para ter um desempenho ideal em relação a não usar corte, e é por isso que recomendamos 0,8 por padrão.

### Privacidade diferencial

O TFF também tem suporte à agregação com privacidade diferencial usando corte adaptativo e ruído gaussiano. É possível construir um agregador para fazer o cálculo das médias com privacidade diferencial da seguinte forma:

In [ ]:
dp_mean = tff.aggregators.DifferentiallyPrivateFactory.gaussian_adaptive(
    noise_multiplier=0.1, clients_per_round=100)

# Equivalent to:
# dp_mean = tff.learning.dp_aggregator(
#   noise_multiplier=0.1, clients_per_round=100, zeroing=False)

Veja as orientações de como definir o argumento `noise_multiplier` no [tutorial sobre privacidade diferencial do TFF](https://www.tensorflow.org/federated/tutorials/federated_learning_with_differential_privacy).

### Compressão com perda

Em comparação à compressão sem perda, como gzip, a compressão com perda costuma resultar em uma taxa de compressão muito maior e ainda pode ser combinada com compressão sem perda posteriormente. Como é necessário gastar menos tempo na comunicação cliente para servidor, as rodadas de treinamento são concluídas mais rápido. Devido à natureza inerentemente aleatória dos algoritmos de aprendizado, até certo limiar, a inexatidão da compressão com perda não tem impacto negativo no desempenho geral.

A recomendação padrão é usar uma quantização uniforme simples (confira [Suresh et al.](http://proceedings.mlr.press/v70/suresh17a/suresh17a.pdf), por exemplo), parametrizada por dois valores: o `threshold` de compressão do tamanho do tensor e o número de `quantization_bits`. Para cada tensor `t`, se o número de elementos de `t` for menor ou igual a `threshold`, ele não é comprimido. Se for maior, os elementos de `t` são quantizados usando arredondamento aleatório para `quantizaton_bits` bits. Ou seja, aplicamos a operação:

`t = round((t - min(t)) / (max(t) - min(t)) * (2**quantizaton_bits - 1)),`

que resulta em valores inteiros no intervalo `[0, 2**quantizaton_bits-1]`. Os valores quantizados são empacotados diretamente em um tipo inteiro para a transmissão e, depois, a transformação inversa é aplicada.

Recomendamos definir `quantizaton_bits` como 8 e `threshold` como 20.000:


In [ ]:
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))

# Equivalent to:
# compressed_mean = tff.learning.compression_aggregator(zeroing=False, clipping=False)

#### Sugestões de ajuste

Ambos os parâmetros `quantization_bits` e `threshold` podem ser ajustados, e o número de clientes que participam de cada rodada de treinamento também pode impactar a eficácia da compressão.

**Limiar.** O valor padrão de 20.000 é escolhido porque observamos que as variáveis com um número pequeno de elementos, como bias em tipos de camada comuns, têm sensibilidade muito maior ao ruído introduzido. Além disso, há poucas vantagens de comprimir variáveis com um número pequeno de elementos na prática, pois o tamanho não comprimido já é relativamente pequeno.

Em algumas aplicações, pode fazer sentido alterar o valor do limiar. Por exemplo: os bias da camada de saída de um modelo de classificação podem ter maior sensibilidade ao ruído. Se você estiver treinando um modelo de linguagem com um vocabulário de 20.004, talvez prefira definir `threshold` como 20.004.

**Bits de quantização.** O valor padrão de 8 para `quantization_bits` deverá ser bom para a maioria dos usuários. Se 8 estiver funcionando bem e você quiser melhorar o desempenho um pouco, pode tentar reduzir para 7 ou 6. Se os recursos permitirem fazer uma pequena pesquisa de grade, recomendamos que você identifique o valor para o qual o treinamento fica instável ou a qualidade do modelo final começa a degradar e então aumente o valor em 2. Por exemplo, se definir `quantization_bits` como 5 funcionar, mas definir como 4 degradar o modelo, recomendamos definir como 6 por precaução.

**Clientes por rodada.** Observe que aumentar consideravelmente o número de clientes por rodada pode permitir que um valor menor de `quantization_bits` funcione bem, pois a inexatidão aleatória introduzida pela quantização pode ser equilibrada pelo cálculo das médias com mais atualizações de clientes.

### Agregação segura

Agregação segura (SecAgg) refere-se a um protocolo criptográfico em que as atualizações de clientes são criptografadas de forma que o servidor só consiga descriptografar a soma. Se o número de clientes que retornarem for insuficiente, o servidor não aprenderá nada. Além disso, o servidor jamais conseguirá inspecionar atualizações individuais. Isso é feito usando-se o operador `tff.federated_secure_sum_bitwidth`.

As atualizações do modelo são valores de ponto flutuante, mas SecAgg faz operações com inteiros. Portanto, precisamos cortar valores grandes para um determinado limite antes de fazer a discretização para um tipo inteiro. O limite de corte pode ser uma constante ou pode ser determinado de forma adaptativa (o padrão recomendado). Os inteiros são então somados de forma segura, e a soma é mapeada de volta para o domínio de ponto flutuante.

Para computar a média com valores ponderados somados usando-se SecAgg com `MY_SECAGG_BOUND` como limite de corte, passe `SecureSumFactory` para `MeanFactory` como:

```
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(MY_SECAGG_BOUND))
```

Para fazer a mesma coisa, mas também determinar os limites de forma adaptativa:

In [ ]:
secagg_bound = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=50.0,
    target_quantile=0.95,
    learning_rate=1.0,
    multiplier=2.0)
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(secagg_bound))

# Equivalent to:
# secure_mean = tff.learning.secure_aggregator(zeroing=Fasle, clipping=False)

#### Sugestões de ajuste

Os parâmetros adaptativos foram escolhidos de forma que os limites sejam estritos (não perderemos muita precisão na discretização), mas o corte ocorra raramente.

Ao ajustar os parâmetros, lembre-se de que o protocolo SecAgg soma as atualizações ponderadas do modelo após ponderar a média. Tipicamente, os pesos são o número de pontos de dados processados localmente e, portanto, para diferentes tarefas, o limite certo pode depender dessa quantidade.

Não recomendamos usar o argumento palavra-chave `increment` ao criar um `secagg_bound` adaptativo, pois isso resultaria em uma perda de precisão relativa grande caso a estimativa real acabe sendo pequena.

O trecho de código acima usará SecAgg somente nos valores ponderados. Se SecAgg também deve ser usado para a soma dos pesos, recomendamos que os limites sejam definidos como constantes da mesma forma que em uma configuração comum de treinamento. O maior peso possível será conhecido de antemão:

```
secure_mean = tff.aggregators.MeanFactory(
    value_sum_factory=tff.aggregators.SecureSumFactory(secagg_bound),
    weight_sum_factory=tff.aggregators.SecureSumFactory(
        upper_bound_threshold=MAX_WEIGHT, lower_bound_threshold=0.0))
```

## Combinação de técnicas

As técnicas individuais para estender uma média apresentadas acima podem ser combinadas.

Recomendamos que a ordem abaixo seja usada ao aplicar as técnicas em clientes:

1. Zeragem
2. Corte
3. Outras técnicas

Os agregadores no módulo `tff.aggregators` são compostos por meio do encapsulamento de "agregadores internos" (cujos efeitos de pré-agregação ocorrem por último, e os efeitos de pós-agregação ocorrem primeiro) dentro de "agregadores externos". Por exemplo: para fazer a zeragem, o corte e a compressão (nessa ordem), deve-se escrever:

```
# Compression is innermost because its pre-aggregation effects are last.
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))
# Compressed mean is inner aggregator to clipping...
clipped_compressed_mean = tff.aggregators.clipping_factory(
    clipping_norm=MY_CLIPPING_CONSTANT,
    inner_agg_factory=compressed_mean)
# ...which is inner aggregator to zeroing, since zeroing happens first.
final_aggregator = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=clipped_compressed_mean)
```

Observe que essa estrutura coincide com os [agregadores padrão](https://github.com/tensorflow/federated/blob/11e4f632b38745c9b38cc39fa1fe67771c206e77/tensorflow_federated/python/learning/model_update_aggregator.py) para algoritmos de aprendizado.

Outras combinações também são possíveis. Vamos estender este documento quando pudermos fornecer uma configuração padrão que funcione em diversas aplicações diferentes. Para implementar novas ideias, confira o tutorial [Implementando agregadores personalizados](custom_aggregators.ipynb).